# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import matplotlib.pyplot as plt #The matplotlib visuailzation library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'week52change': 1.299588,
 'week52high': 457.18,
 'week52low': 197.82,
 'marketcap': 1930648068246,
 'employees': 140008,
 'day200MovingAvg': 314.51,
 'day50MovingAvg': 368.79,
 'float': 4484763860,
 'avg10Volume': 46933959.2,
 'avg30Volume': 38158022.1,
 'ttmEPS': 13.8486,
 'ttmDividendRate': 3.23,
 'companyName': 'Apple, Inc.',
 'sharesOutstanding': 4380957803,
 'maxChangePercent': 453.609,
 'year5ChangePercent': 2.9033,
 'year2ChangePercent': 1.1424,
 'year1ChangePercent': 1.318828,
 'ytdChangePercent': 0.4609,
 'month6ChangePercent': 0.372753,
 'month3ChangePercent': 0.49576,
 'month1ChangePercent': 0.180563,
 'day30ChangePercent': 0.203278,
 'day5ChangePercent': 0.155289,
 'nextDividendDate': '2020-07-26',
 'dividendYield': 0.00733941424618713,
 'nextEarningsDate': '2020-10-28',
 'exDividendDate': '2020-05-03',
 'peRatio': 33.68,
 'beta': 1.1464714088617771}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course.

In [10]:
data['year1ChangePercent']

1.318828

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project.

In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])



Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['stats']['year1ChangePercent']
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

